In [11]:
import os
import sys
import yaml # type: ignore
from glob import glob

import torch

import sys
sys.path.append("/kaggle/src/")

from conf import ExpConfig
from model.model_module import ModelModule
from data.make_maxmean_df import save_centerslice_maxmean_df

from score.compute_score import compute_surface_dice_score, add_size_columns

In [12]:
# TEST_DATA_DIR = "/kaggle/input/blood-vessel-segmentation/test"
# TEST_DATA_DIR = "/kaggle/input/blood-vessel-segmentation/test_debug"

# test_imgdir_list = glob(os.path.join(TEST_DATA_DIR, "*", "images"))
# test_datapath_list = [os.path.dirname(data_dir) for data_dir in test_imgdir_list]
# TEST_DATANAME_LIST = [os.path.split(data_dir)[-1] for data_dir in test_datapath_list]


TEST_DATA_DIR = "/kaggle/input/blood-vessel-segmentation/train"
TEST_DATANAME_LIST = ["kidney_2"]

print(TEST_DATANAME_LIST)

['kidney_2']


In [13]:
exps_dir = "/kaggle/working/weights"
# exp_name = "exp003_Gradloss"
# exp_name = "exp002_seresnext50_32x4d_Gradloss05"
exp_name = "exp006_1024"
model_path = os.path.join(exps_dir, exp_name, "best.pth")
config_path = os.path.join(exps_dir, exp_name, "config.yaml")

config_dict = yaml.load(open(config_path, "r"), Loader=yaml.SafeLoader)
config = ExpConfig(**config_dict)

config.overlap_rate = 0.2

print(config)

ExpConfig(debug=False, phase='train', exp_name='exp006_1024', exp_category='baseline', seed=42, model_name='SegModel', encoder_name='seresnext50_32x4d', pretrained=True, in_channels=1, out_channels=1, use_batchnorm=True, dropout=0.2, encoder_channels=[64, 128, 256, 512, 512], decoder_channels=[512, 256, 128, 64, 64], input_dir='/kaggle/input', competition_name='blood-vessel-segmentation', input_data_dir='/kaggle/input/blood-vessel-segmentation', processed_data_dir='/kaggle/working/_processed', output_dir='/kaggle/working', save_dir='/kaggle/working/exp006_1024', img_height=1024, img_width=1024, stride_height=1024, stride_width=1024, patch_height=1536, patch_width=1536, slice_num=1, batch_size=2, num_workers=2, train_df='/kaggle/working/train_1024_1024.csv', valid_df='/kaggle/working/valid_1024_1024.csv', train_data_name=['kidney_1_dense'], valid_data_name=['kidney_2'], minmax_df_path='/kaggle/working/centerslice_maxmean.csv', epochs=10, T_max=10, lr=0.0001, eta_min=1e-06, loss_type='di

In [14]:
# df = save_centerslice_maxmean_df(data_dir=TEST_DATA_DIR,
#                                  save_dir="/kaggle/working",
#                                  use_data_ratio = 0.1) 
# display(df)

In [15]:
model = ModelModule(config, phase="test")
# modelの読み込み
print(model_path)
model.load_state_dict(torch.load(model_path))
model.to("cuda")
print("model loaded")


/kaggle/working/weights/exp006_1024/best.pth
model loaded


In [16]:
submit = model.inference(TEST_DATA_DIR, TEST_DATANAME_LIST)


 predicting... => kidney_2, slice num: 2217
2200/2217, 26.52 [min]

In [17]:
submit

,id,rle
0,kidney_2_0000,1 0
1,kidney_2_0001,1 0
2,kidney_2_0002,1 0
3,kidney_2_0003,1 0
4,kidney_2_0004,1 0
...,...,...
2212,kidney_2_2212,1 0
2213,kidney_2_2213,1 0
2214,kidney_2_2214,1 0
2215,kidney_2_2215,1 0


In [18]:
import pandas as pd

data_id_list = submit["id"].values.tolist()

label = pd.read_csv("/kaggle/input/blood-vessel-segmentation/train_rles.csv")
label = label[label["id"].isin(data_id_list)].reset_index(drop=True)
add_size_columns(label)

In [19]:
label

,id,rle,width,height,image_id,slice_id
0,kidney_2_0000,1 0,1511,1041,kidney_2,0000
1,kidney_2_0001,1 0,1511,1041,kidney_2,0001
2,kidney_2_0002,1 0,1511,1041,kidney_2,0002
3,kidney_2_0003,1 0,1511,1041,kidney_2,0003
4,kidney_2_0004,1 0,1511,1041,kidney_2,0004
...,...,...,...,...,...,...
2212,kidney_2_2212,1 0,1511,1041,kidney_2,2212
2213,kidney_2_2213,1 0,1511,1041,kidney_2,2213
2214,kidney_2_2214,1 0,1511,1041,kidney_2,2214
2215,kidney_2_2215,1 0,1511,1041,kidney_2,2215


In [20]:
%%time
# Compute surface Dice score
score = compute_surface_dice_score(submit, label)
score

CPU times: user 4.58 s, sys: 4.01 ms, total: 4.59 s
Wall time: 4.59 s


2.680524266907014e-05